In [9]:
import torch
import numpy as np
from typing import Dict, List, Optional
import json
import os
from scipy.stats import zscore
from datetime import datetime
import uuid

from eeg_assessment import save_to_json, analyze_patient_eeg, generate_synthetic_eeg_data

In [11]:
"""Main entry point for running EEGPT patient assessment workflow."""

voltage_data, sampling_rate, channel_names = generate_synthetic_eeg_data()

assessment = run_patient_assessment(voltage_data, sampling_rate, channel_names)
report = format_patient_assessment(assessment)
print(report)

save_path = f'data/patient_eeg_assessment-{uuid.uuid4()}.json'
save_to_json(assessment, save_path)
print("\n✓ Complete assessment saved to 'patient_eeg_assessment.json'")


Loading pretrained EEGPT model...
✓ Pretrained EEGPT model loaded successfully
✓ EEGPT Patient Assessment initialized on cpu
✓ Using pretrained model: checkpoint/eegpt_mcae_58chs_4s_large4E.ckpt
Analyzing EEG: (8, 1000) channels, 1000 samples
Duration: 4.0 seconds
EEGPT Patient Assessment System

PATIENT ASSESSMENT RESULTS:
------------------------------
Overall State: Normal Brain Activity
Alertness: 6.1/10
Stress Level: 5.1/10
Attention: 4.8/10
Cognitive State: Normal
Seizure Risk: Low Risk
Confidence: 88.0%

CLINICAL INTERPRETATIONS:
------------------------------
Alertness: Good alertness - Normal waking consciousness and attention
Stress: Moderate stress - Within normal range for daily activities
Attention: Moderate attention - Some difficulty with sustained concentration
Cognitive: Normal (confidence: 33.8%)
Seizure_Risk: Low Risk (confidence: 30.8%)

RECOMMENDATIONS:
------------------------------
1. Normal findings - routine monitoring sufficient

MOST IMPORTANT CHANNELS:
F3, F